In [7]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm




In [8]:
data1 = np.load('data1.npy')
lab1 = np.load('lab1.npy')
data0 = np.load('data0.npy')
lab0 = np.load('lab0.npy')
data2 = np.load('data2.npy')
lab2 = np.load('lab2.npy')
data = np.concatenate([data0, data1, data2], axis=0)
labels = np.concatenate([lab0, lab1, lab2], axis=0)
print(f'Data shape: {data.shape}')
print(f'Labels shape: {labels.shape}')

Data shape: (30000, 40, 168)
Labels shape: (30000,)


In [9]:
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            sample = self.transform(sample)

        return torch.tensor(sample, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Repeat the single channel 3 times
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

dataset = CustomDataset(data, labels, transform=transform)

loader = DataLoader(dataset, batch_size=1024, shuffle=True)



In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=False) 
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 37)

model = model.to(device)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs for loading!")
    model = nn.DataParallel(model)

model.load_state_dict(torch.load("part2.pth", map_location=device))
model.eval() 

print("Model loaded successfully!")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    print(f"Accuracy: {100 * correct / total:.2f}%")

model = model.to(device)

evaluate_model(model, loader)

Using 4 GPUs for loading!
Model loaded successfully!


/tmp/ipykernel_7721/1044777366.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("part2.pth", map_location=device))
/tmp/ipykernel_7721/7

Accuracy: 93.91%
